In [248]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
import random
%matplotlib inline

In [249]:
# read in all the words
words = open('data/names.txt', 'r').read().splitlines()
print(words[:10])

# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)

print(f'Character to index mapping: {itos}')
print(f'Vocabulary size: {vocab_size}')

# Shuffle the words
random.seed(42)
random.shuffle(words)


['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']
Character to index mapping: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Vocabulary size: 27


In [252]:
def encode_words(words):
    encoded = []
    for w in words:
        encoded.extend([stoi[ch] for ch in '.' + w ])
    return encoded

encoded = encode_words(words)

In [253]:
encoded[:20]

[0, 25, 21, 8, 5, 14, 7, 0, 4, 9, 15, 14, 4, 18, 5, 0, 24, 1, 22, 9]

In [254]:
n = len(encoded)
n1 = int(0.8 * n)
block_size = 8
batch_size = 32

train_seq = encoded[:n1]
dev_seq = encoded[n1:]

In [255]:
def create_pairs(seq, block_size):
    X, Y = [], []
    for i in range(0, len(seq) - block_size):
        X.append(seq[i:i+block_size])
        Y.append(seq[i+1:i+block_size+1])
    X = torch.tensor(X, dtype=torch.float32)
    Y = torch.tensor(Y, dtype=torch.float32)
    return X, Y

Xtr, Ytr = create_pairs(train_seq, block_size)
Xdev, Ydev = create_pairs(dev_seq, block_size)

In [256]:
print(Xtr[1])
print(Ytr[1])
print(f'Training data shapes - X: {Xtr.shape}, Y: {Ytr.shape}')
print(f'Development data shapes - X: {Xdev.shape}, Y: {Ydev.shape}')

tensor([25., 21.,  8.,  5., 14.,  7.,  0.,  4.])
tensor([21.,  8.,  5., 14.,  7.,  0.,  4.,  9.])
Training data shapes - X: torch.Size([182508, 8]), Y: torch.Size([182508, 8])
Development data shapes - X: torch.Size([45622, 8]), Y: torch.Size([45622, 8])


In [260]:
def split_into_batches(X, Y, batch_size, time_steps):
    num_batches = X.size(0) // batch_size
    
    # Adjust the number of examples to be divisible by batch_size * time_steps
    num_examples = num_batches * batch_size
    X = X[:num_examples, :]
    Y = Y[:num_examples, :]

    # Reshape X and Y to have dimensions: (time_steps, num_batches, batch_size)
    X = X.view(time_steps, num_batches, batch_size).permute(0, 1, 2)
    Y = Y.view(time_steps, num_batches, batch_size).permute(0, 1, 2)

    return X, Y


Xtr_batched, Ytr_batched = split_into_batches(Xtr, Ytr, 32, 8)
Xdev_batched, Ydev_batched = split_into_batches(Xdev, Ydev, 32, 8)


print(f'Training batches shape - X: {Xtr_batched.shape}, Y: {Ytr_batched.shape}')
print(f'Development batches shape - X: {Xdev_batched.shape}, Y: {Ydev_batched.shape}')

# (time_steps, num_batches, batch_size)

# print(Xtr_batched.shape)

# print(Xtr_batched[:, 1, 0:10])
# print("\n")
# print(Ytr_batched[:, 1, 0:10])
# print("\n")

# print("next batch")
# print(Xtr_batched[:, 0, 26:32])
# print("\n")
# print(Ytr_batched[:, 0, 26:32])



# (8, 1, 32) -> (8, 32)

Training batches shape - X: torch.Size([8, 5703, 32]), Y: torch.Size([8, 5703, 32])
Development batches shape - X: torch.Size([8, 1425, 32]), Y: torch.Size([8, 1425, 32])


In [292]:
batch_size = 1
hidden_size = 30
time_steps = 8
input_size = 27

# Parameters
Fvh = torch.randn(vocab_size, hidden_size)
i1vh = torch.randn(vocab_size, hidden_size)
i2vh = torch.randn(vocab_size, hidden_size)
Ovh = torch.randn(vocab_size, hidden_size)

Fhh = torch.randn(hidden_size, hidden_size)
i1hh = torch.randn(hidden_size, hidden_size)
i2hh = torch.randn(hidden_size, hidden_size)
Ohh = torch.randn(hidden_size, hidden_size)

bias1 = torch.zeros(hidden_size)
bias2 = torch.zeros(hidden_size)
bias3 = torch.zeros(hidden_size)
bias4 = torch.zeros(hidden_size)

output_matrix = torch.randn(hidden_size, vocab_size)

# Storage
preact1 = torch.zeros(time_steps, batch_size, hidden_size)
preact2 = torch.zeros(time_steps, batch_size, hidden_size)
preact3 = torch.zeros(time_steps, batch_size, hidden_size)
preact4 = torch.zeros(time_steps, batch_size, hidden_size)

act1 = torch.zeros(time_steps, batch_size, hidden_size)
act2 = torch.zeros(time_steps, batch_size, hidden_size)
act3 = torch.zeros(time_steps, batch_size, hidden_size)
act4 = torch.zeros((time_steps, batch_size, hidden_size))

Cin = torch.zeros((time_steps, batch_size, hidden_size))
Cout = torch.zeros((time_steps, batch_size, hidden_size))
Ctout = torch.zeros((time_steps, batch_size, hidden_size))

Hin = torch.zeros((time_steps, batch_size, hidden_size))
Hout = torch.zeros((time_steps, batch_size, hidden_size))

logits = torch.zeros((time_steps, batch_size, vocab_size))

c0 = torch.zeros(batch_size, hidden_size)
h0 = torch.zeros((batch_size, hidden_size))

# Backward pass

# To update
dFvh = torch.zeros(vocab_size, hidden_size) / 0.0
di1vh = torch.zeros(vocab_size, hidden_size) / 0.0
di2vh = torch.zeros(vocab_size, hidden_size) / 0.0
dOvh = torch.zeros(vocab_size, hidden_size) / 0.0

dFhh = torch.zeros(hidden_size, hidden_size)/ 0.0
di1hh = torch.zeros(hidden_size, hidden_size)/ 0.0
di2hh = torch.zeros(hidden_size, hidden_size)/ 0.0
dOhh = torch.zeros(hidden_size, hidden_size)/ 0.0

dbias1 = torch.zeros(hidden_size) # (30)
dbias2 = torch.zeros(hidden_size) # (30)
dbias3 = torch.zeros(hidden_size) # (30)
dbias4 = torch.zeros(hidden_size) # (30)

doutput_matrix = torch.randn(hidden_size, vocab_size) # (30, 27)


# Placeholders (indexed with t)
dlogits = torch.zeros((time_steps, batch_size, vocab_size)) # (30, 27)
dhidden1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dhidden2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dtotal = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)

dpreact1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30) 
dpreact2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dpreact3 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dpreact4 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)

dact1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30) 
dact2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dact3 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dact4 = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)

dC = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dCt = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dHin = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dHout = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dlogits = torch.zeros((time_steps, batch_size, vocab_size)) # (32, 27)

dc0 = torch.zeros(batch_size, hidden_size)
dh0 = torch.zeros((batch_size, hidden_size))
dcn = torch.zeros(batch_size, hidden_size)
dhn = torch.zeros((batch_size, hidden_size))

In [289]:
lr = 0.8

In [291]:
while True:
    # Start with no cell activations, and no hidden activations
    h0 = torch.zeros((batch_size, hidden_size))
    c0 = torch.zeros((batch_size, hidden_size))
    
    # Loop over all batches (6504 total)
    for batch_num in reversed(range(Xtr.shape[1])):
        # Get a batch of random numbers into correct shape
        Xb = Xtr_batched[:, batch_num, :] # ( 8, 32)
        Xb = Xb.to(torch.long)
        Xb = F.one_hot(Xb, 27) # (8, 32, 27)
        Xb = Xb / 1.0
        Yb = Ytr_batched[batch_num]
        Yb = Yb.to(torch.long)
    
        # Forward propagation
        for t in range(time_steps):
            # if c0 is None: c0 = np.zeros((b,d))
            # if h0 is None: h0 = np.zeros((b,d))
            if t == 0:
                Hin[t] = h0
                Cin[t] = c0
            else:
                Hin[t] = Hout[t-1]
                Cin[t] = Cout[t-1]
            loss = 0
            
            preact1[t] = Xb[t] @ Fvh + Hin[t] @ Fhh + bias1 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
            preact2[t] = Xb[t] @ i1vh + Hin[t] @ i1hh * bias2 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
            preact3[t] = Xb[t] @ i2vh + Hin[t] @ i2hh + bias3 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
            preact4[t] = Xb[t] @ Ovh + Hin[t] @ Ohh + bias4 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
            
            act1[t] = torch.sigmoid(preact1[t]) # (32, 30)
            act2[t] = torch.sigmoid(preact2[t]) # (32, 30)
            act3[t] = torch.tanh(preact3[t]) # (32, 30)
            act4[t] = torch.sigmoid(preact4[t]) # (32, 30)
            
            Cout[t] = act1[t] * Cin[t] + act2[t] * act3[t] # (32, 30)
            if t < time_steps -1: Cin[t+1] = Cout[t]
            Ctout[t] = torch.tanh(Cout[t]) # (32, 30)
            Hout[t] = Ctout[t] * act4[t] # (32, 30)
            if t < time_steps -1: Hin[t+1] = Hout[t] 
            
            
            logits[t] = Hout[t] @ output_matrix # (32, 27)
            counts = logits.exp()
            counts_sum = counts.sum(1, keepdims=True)
            counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
            probs = counts * counts_sum_inv
            logprobs = probs.log()
            loss += -logprobs[t][torch.arange(32), Yb].mean()
    
        if (batch_num % 100 == 0):
            print (loss / time_steps)
    
        # h0 = Hout[0]
        # c0 = Cout[0]
        h0 = torch.zeros((batch_size, hidden_size))
        c0 = torch.zeros((batch_size, hidden_size))
        
    ### ------------------------------------------------------------------------------------------------------------------
        # Backward pass
        for t in reversed(range(time_steps)):
        
            # Backpropogate cross entropy
            dlogits[t] = F.softmax(logits[t], 1)
            dlogits[t][torch.arange(batch_size), Yb] -= 1
            dlogits /= n
            
            # Backpropogate dHout
            # t = 7, 6, 5, 4, 3, 2, 1, 0
            if (t < time_steps-1):
                # dHout of a previous time step, must add dHin of the next time step
                # dHout of one time step, becomes the input for the next time step
                # Hout[t] = Hin[t+1]
                dHout[t] = dHin[t+1]
                dHout[t] =  dlogits[t] @ output_matrix.T + dHin[t+1] # (32, 27) @ (27, 30) = (32, 30) dHt on paper derivation
            else: # t = 7
                dHout[t] =  dlogits[t] @ output_matrix.T + dhn
        
            # Backpropogate output matrix
            doutput_matrix = dlogits[t].T @ dHout[t] # (32, 27)
            
            # Backpropogate dact3 (output gate activations)
            dact3[t] = dHout[t] * Ctout[t] # (32, 27) * (32, 27) = (32, 27)
            
            # Backpropogate dC (current cell state)
            dC[t] = dHout[t] * act4[t] * (1 - torch.tanh(Cout[t])**2) # (32, 27) * (32, 27) * (32, 27) = (32, 27)
            
            # Backpropogate act1 and previous cell state
            if t > 0:
                # Forget gate activations
                # Last cell activations
                dact1[t] = dC[t] * Cout[t-1] # (32, 27) * (32, 27) = (32, 27)
                dC[t-1] = dC[t] * act1[t]
            else:
                dact1[t] = dC[t] * c0
                dc0 = dC[t] * act1[t]
            
            # Backpropogate i1 activations
            dact2[t] = dC[t] * act3[t]
            
            # Backpropogate i2 activations
            dact3[t] = dC[t] * act2[t]
            
            # Backpropogate all preactivations
            dpreact1[t] = dact1[t] * act1[t] * ( 1- act1[t])
            dpreact2[t] = dact2[t] * act2[t] * ( 1- act2[t])
            dpreact3[t] = dact3[t] * (1 - torch.tanh(dpreact3[t])**2)
            dpreact4[t] = dact4[t] * act4[t] * ( 1- act4[t])
            
            # Backpropogate gates
            dFvh = Xb[t].T @ dpreact1[t] # (27, 32) (32, 30) = (27, 30)
            dFhh = Hin[t].T @ dpreact1[t] 
            di1vh = Xb[t].T @ dpreact2[t]
            di1hh = Hin[t].T @ dpreact2[t]
            di2vh = Xb[t].T @ dpreact3[t]
            di2hh = Hin[t].T @ dpreact3[t]
            dOvh = Xb[t].T @ dpreact4[t]
            dOhh = Hin[t].T @ dpreact4[t]
            
            # Backpropogate prevh
            dHin[t] = dpreact1[t] @ Fhh.T + dpreact2[t] @ i1hh.T + dpreact3[t] @ i2hh.T + dpreact4[t] @ Ohh.T
    
    ### ------------------------------------------------------------------------------------------------------------------
    
        # Update parameters using gradients
        Fvh -= lr * dFvh
        i1vh -= lr * di1vh
        i2vh -= lr * di2vh
        Ovh -= lr * dOvh
        
        Fhh -= lr * dFhh
        i1hh -= lr * di1hh
        i2hh -= lr * di2hh
        Ohh -= lr * dOhh
        
        bias1 -= lr * dbias1
        bias2 -= lr * dbias2
        bias3 -= lr * dbias3
        bias4 -= lr * dbias4
    

tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5434)
tensor(0.5433)
tensor(0.5433)
tensor(0.5433)
tensor(0.5432)
tensor(0.5432)
tensor(0.5432)
tensor(0.5431)
tensor(0.5431)
tensor(0.5431)
tensor(0.5430)
tensor(0.5430)
tensor(0.5429)
tensor(0.5429)
tensor(0.5429)
tensor(0.5428)
tensor(0.5428)
tensor(0.5427)
tensor(0.5427)
tensor(0.5426)
tensor(0.5426)
tensor(0.5425)
tensor(0.5425)
tensor(0.5424)
tensor(0.5424)
tensor(0.5423)
tensor(0.5423)
tensor(0.5422)
tensor(0.5422)
tensor(0.5421)
tensor(0.5421)
tensor(0.5420)
tensor(0.5420)
tensor(0.5419)
tensor(0.5418)
tensor(0.5418)


KeyboardInterrupt: 

In [272]:
import torch

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def softmax(x):
    e_x = torch.exp(x - torch.max(x, dim=1, keepdim=True)[0])
    return e_x / torch.sum(e_x, dim=1, keepdim=True)

def sample_model(start_vector, Fvh, Fhh, i1vh, i1hh, i2vh, i2hh, Ovh, Ohh, bias1, bias2, bias3, bias4, output_matrix):
    """
    Generate a sequence of samples from the model.
    
    Parameters:
    - start_vector: Initial one-hot encoded vector to start the generation (torch tensor of shape (1, input_size))
    - Other parameters: Model weights and biases
    
    Returns:
    - Generated sequence (list of integers)
    """
    input_size = start_vector.shape[1]
    hidden_size = Fvh.shape[1]
    
    # Initialize hidden and cell states
    h = torch.zeros((1, hidden_size))
    c = torch.zeros((1, hidden_size))
    
    # Initialize the generated sequence with the index of the start vector
    generated_sequence = [torch.argmax(start_vector).item()]
    
    # Loop until '.' is generated or the maximum sequence length is reached
    while generated_sequence[-1] != 0 and len(generated_sequence) <= max_length:
        x = start_vector  # Use the provided start vector as input
        
        preact1 = x @ Fvh + h @ Fhh + bias1
        preact2 = x @ i1vh + h @ i1hh + bias2
        preact3 = x @ i2vh + h @ i2hh + bias3
        preact4 = x @ Ovh + h @ Ohh + bias4
        
        act1 = torch.sigmoid(preact1)
        act2 = torch.sigmoid(preact2)
        act3 = torch.tanh(preact3)
        act4 = torch.sigmoid(preact4)
        
        c = act1 * c + act2 * act3
        h = torch.tanh(c) * act4
        
        logits = h @ output_matrix
        probs = softmax(logits)
        
        # Sample from the probability distribution to get the next input
        next_index = torch.multinomial(probs.squeeze(), 1).item()
        
        generated_sequence.append(next_index)
        
        # Update the start vector with the one-hot encoded representation of the next index
        start_vector = torch.zeros((1, input_size))
        start_vector[0, next_index] = 1
        
    return [itos[ch] for ch in generated_sequence]

# Example usage
input_size = 27  # Size of the one-hot encoded vector
start_index = 1  # Index representing the starting point (e.g., 'a')
start_vector = torch.zeros((1, input_size))
start_vector[0, start_index] = 1  # One-hot encode the starting point

# Assume Fvh, Fhh, i1vh, i1hh, i2vh, i2hh, Ovh, Ohh, bias1, bias2, bias3, bias4, and output_matrix are already defined

max_length = 100  # Maximum length of the generated sequence


In [288]:
generated_sequence = sample_model(start_vector, Fvh, Fhh, i1vh, i1hh, i2vh, i2hh, Ovh, Ohh, bias1, bias2, bias3, bias4, output_matrix)
print(''.join(generated_sequence))

axypdouqtzn.
